In [10]:
import numpy as np
import pandas as pd

In [11]:
train = pd.read_csv('DATOS_PROYECTO_4/train_emoji.csv',header=None)
test = pd.read_csv('DATOS_PROYECTO_4/test_emoji.csv',header=None)

In [12]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [13]:
import emoji as emoji

In [14]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [15]:
emoji.emojize(":fork_and_knife:")

'🍴'

In [16]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [17]:
data = train.values
for i in range(10):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [18]:
from tensorflow.keras.utils import to_categorical

In [19]:
XT = train[0]
Xt = test[0]

YT = to_categorical(train[1])
Yt = to_categorical(test[1])


print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [20]:
embeddings = {}
with open('Glove_Embeddings/glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word] = coeffs

In [21]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [22]:
XT[1]

'I am proud of your achievements'

In [23]:
emb_XT = getOutputEmbeddings(XT)
emb_Xt = getOutputEmbeddings(Xt)

C:\Users\USER\AppData\Local\Temp/ipykernel_3816/2176504226.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix] = X[ix].split()


In [24]:
print(emb_XT.shape)
print(emb_Xt.shape)


(132, 10, 50)
(56, 10, 50)


In [25]:
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.models import Sequential

In [26]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 64)            29440     
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
____________________________________________________

In [27]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(emb_XT,YT,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

Epoch 1/40
4/4 [==============================] - 3s 190ms/step - loss: 1.6132 - acc: 0.2288 - val_loss: 1.6026 - val_acc: 0.2143
Epoch 2/40
4/4 [==============================] - 0s 17ms/step - loss: 1.5546 - acc: 0.3475 - val_loss: 1.6030 - val_acc: 0.2857
Epoch 3/40
4/4 [==============================] - 0s 17ms/step - loss: 1.4916 - acc: 0.4068 - val_loss: 1.6150 - val_acc: 0.2857
Epoch 4/40
4/4 [==============================] - 0s 16ms/step - loss: 1.4620 - acc: 0.4237 - val_loss: 1.6181 - val_acc: 0.3571
Epoch 5/40
4/4 [==============================] - 0s 15ms/step - loss: 1.4267 - acc: 0.4153 - val_loss: 1.5772 - val_acc: 0.2857
Epoch 6/40
4/4 [==============================] - 0s 16ms/step - loss: 1.4029 - acc: 0.3898 - val_loss: 1.5247 - val_acc: 0.1429
Epoch 7/40
4/4 [==============================] - 0s 15ms/step - loss: 1.2956 - acc: 0.5424 - val_loss: 1.4180 - val_acc: 0.2143
Epoch 8/40
4/4 [==============================] - 0s 16ms/step - loss: 1.2358 - acc: 0.5508 - va

In [28]:
model.evaluate(emb_Xt,Yt)

2/2 [==============================] - 0s 3ms/step - loss: 1.5921 - acc: 0.5893


[1.592056155204773, 0.5892857313156128]

In [29]:
pred = model.predict_classes(emb_Xt)

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [ ]:
for i in range(30):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
😁
she got me a present
❤️
😁
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
😁
I am upset
❤️
⚾
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
🍴
Stop making this joke ha ha ha
😁
😁
where is the ball
⚾
⚾
work is hard
😓
😁
This girl is messing with me
😓
❤️
are you serious ha ha
😁
😓
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😓
Congratulation for having a baby
😁
😁
stop messing around
😓
😓
any suggestions for dinner
🍴
🍴
I love taking breaks
❤️
❤️
you brighten my day
😁
❤️
I boiled rice
🍴
🍴
she is a bully
😓
😁
Why are you feeling bad
😓
😓
I am upset
😓
⚾
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
⚾
valentine day is near
❤️
😁


In [ ]:
with open("model.json", "w") as file:
    file.write(model.to_json())
model.save_weights("model.h5")

### Code to be written in Django.

In [ ]:
from keras.models import model_from_json

In [ ]:
with open("model.json", "r") as file:
    model = model_from_json(file.read())
model.load_weights("model.h5")

In [ ]:
test_str = "Hello how are you"
X = pd.Series([test_str])

In [ ]:
emb_X = getOutputEmbeddings(X)

In [ ]:
p = model.predict_classes(emb_X)

In [ ]:
print(' '.join(X[0]))
print(emoji.emojize(emoji_dictionary[str(p[0])]))

Hello how are you
😁
